In [2]:
import zipfile
import os

def extract_files(zip_path):
    extract_dir = os.path.dirname(zip_path)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Extracted all files from '{zip_path}' into '{extract_dir}'.")

extract_files("fire.zip")

Extracted all files from 'fire.zip' into ''.


In [1]:
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split



In [2]:
# Define transformations for the images
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}



In [3]:
# Define dataset directory
dataset_dir = 'dataset_train'  # Current working directory where 'fire' and 'nofire' folders exist

# Load the dataset
dataset = datasets.ImageFolder(root=dataset_dir, transform=data_transforms['train'])

# Calculate dataset sizes for splits
total_size = len(dataset)
train_size = int(0.6 * total_size)
val_size = int(0.2 * total_size)
test_size = total_size - train_size - val_size

# Split the dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Apply appropriate transforms to validation and test datasets
val_dataset.dataset.transform = data_transforms['val']
test_dataset.dataset.transform = data_transforms['test']

# Create dataloaders
batch_size = 16
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False),
    'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False),
}

In [4]:
# Load ResNet50 model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=False)
num_features = model.fc.in_features
model.fc = torch.nn.Linear(num_features, 2)  # Binary classification
model = model.to(device)

# Define loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/dbhatka/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [6]:
# Training loop
def train_model(model, dataloaders, criterion, optimizer, num_epochs=20):
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        # print('-' * 20)

        # Training phase
        model.train()
        train_loss = 0.0
        train_corrects = 0
        for inputs, labels in dataloaders['train']:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_corrects += torch.sum(preds == labels.data)

        train_loss /= len(dataloaders['train'].dataset)
        train_accuracy = train_corrects.double() / len(dataloaders['train'].dataset)
        print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in dataloaders['val']:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_corrects += torch.sum(preds == labels.data)

        val_loss /= len(dataloaders['val'].dataset)
        val_accuracy = val_corrects.double() / len(dataloaders['val'].dataset)
        print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

    return model


# # Prepare dataloaders for training
# dataloaders = {
#     'train': train_loader,
#     'val': val_loader
# }


# Train the model
trained_model = train_model(model, dataloaders, criterion, optimizer, num_epochs=20)

Epoch 1/20
Train Loss: 0.0003, Train Accuracy: 0.9999
Val Loss: 0.0004, Val Accuracy: 0.9999
Epoch 2/20
Train Loss: 0.0045, Train Accuracy: 0.9988
Val Loss: 0.0002, Val Accuracy: 1.0000
Epoch 3/20
Train Loss: 0.0003, Train Accuracy: 0.9999
Val Loss: 0.0001, Val Accuracy: 1.0000
Epoch 4/20
Train Loss: 0.0018, Train Accuracy: 0.9997
Val Loss: 0.0000, Val Accuracy: 1.0000
Epoch 5/20
Train Loss: 0.0009, Train Accuracy: 0.9997
Val Loss: 0.0000, Val Accuracy: 1.0000
Epoch 6/20
Train Loss: 0.0020, Train Accuracy: 0.9996
Val Loss: 0.0002, Val Accuracy: 0.9999
Epoch 7/20
Train Loss: 0.0024, Train Accuracy: 0.9995
Val Loss: 0.0000, Val Accuracy: 1.0000
Epoch 8/20
Train Loss: 0.0030, Train Accuracy: 0.9995
Val Loss: 0.0000, Val Accuracy: 1.0000
Epoch 9/20
Train Loss: 0.0003, Train Accuracy: 0.9999
Val Loss: 0.0000, Val Accuracy: 1.0000
Epoch 10/20
Train Loss: 0.0006, Train Accuracy: 0.9999
Val Loss: 0.0000, Val Accuracy: 1.0000
Epoch 11/20
Train Loss: 0.0025, Train Accuracy: 0.9994
Val Loss: 0.00

In [7]:
# Test loop
def test_model(model, dataloader):
    model.eval()
    test_corrects = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            test_corrects += torch.sum(preds == labels.data)

    test_accuracy = test_corrects.double() / len(dataloader.dataset)
    print(f"Test Accuracy: {test_accuracy:.4f}")


# Evaluate on test set
print("Testing the model...")
test_model(trained_model, dataloaders['test'])


Testing the model...
Test Accuracy: 1.0000


In [8]:
# for saving the model
model_save_path = "flame2resnet2.pth"
torch.save(trained_model.state_dict(), model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to flame2resnet2.pth


In [ ]:
# for loading the model
from torchvision import models

# Initialize the ResNet50 model
model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 2)  # Assuming binary classification (fire vs nofire)

model_load_path = "flame2resnet.pth"
model.load_state_dict(torch.load(model_load_path))
model.eval()
# model = model.to(device)